#### Imports

In [1]:
# Telegram Bot
import telebot
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton

# Config Driver
from config.config_driver import ConfigDriver

# Amazon Driver
from amazon.amazon import Amazon

# Definitions
from definitions import *

#### Variables Main

In [ ]:
API_TOKEN = 'Your_Token_Here'
bot = telebot.TeleBot(API_TOKEN)

config_driver = ConfigDriver()
driver = config_driver.get_driver()

amazon_config = Amazon(driver)
offers_day = False

#### Main

In [3]:
def prepare_markup():
    """ Method to prepare the markup """
    
    markup = InlineKeyboardMarkup()
    
    option1 = InlineKeyboardButton("Pesquisar Preço de Produto", callback_data="opt1")
    option2 = InlineKeyboardButton("Ofertas do Dia", callback_data="opt2")
    option3 = InlineKeyboardButton("Prime Vídeo", callback_data="opt3")

    markup.add(option1, option2, option3)
        
    return markup

def get_function_details(message):
    """ Method to get the function details """
    
    global offers_day
    
    response = {}
    
    if message.text == '/start':
        response['message'] = "Assistente Virtual da Amazon inicializado!"
    
    elif message.text in SAUDACOES:
        user = message.from_user.first_name
        
        markup = InlineKeyboardMarkup()
        
        response['markup'] = prepare_markup()
        
        response['text'] = f"Eae {user}! Em que posso te ajudar?"
    elif offers_day and amazon_config.is_departament(message.text):
        departament_choice = message.text
        response['products'] = amazon_config.get_products_of_departament(departament_choice)
        offers_day = False
    else:
        response['message'] = "Desculpe, eu não entendi o que você disse."
        
    return response

################################################################################
@bot.message_handler(func=lambda message: True)
def reply_message(message):
    """ Method to reply the message """
    
    response = get_function_details(message)
    
    if 'markup' in response:
        bot.reply_to(message, response['text'], reply_markup=response["markup"])
    elif 'message' in response:
        bot.send_message(message.chat.id, response['message'])
    elif 'products' in response:
        list_products = ()

        for product in response['products']:
            product_name, product_link, product_image, product_discount = product

            if product_name == "" or product_link == "" or product_image == "" or product_discount == "":
                continue
            elif product_name == None or product_link == None or product_image == None or product_discount == None:
                continue

            list_products += (product_name, product_link, product_image, product_discount),

        bot.send_message(message.chat.id, f"Foi encontrado {len(list_products)} produtos!")
        bot.send_message(message.chat.id, "Aqui estão as ofertas do dia!")

        for product in list_products:
            product_name, product_link, product_image, product_discount = product

            message_text = (
                f"Produto: {product_name}\n"
                f"Link: {product_link}\n"
                f"Desconto: {product_discount}"
            )
            
            bot.send_photo(message.chat.id, product_image, caption=message_text)
        bot.send_message(message.chat.id, "O que posso te ajudar?", reply_markup=prepare_markup())
    else:
        bot.reply_to(message, response['text'])

################################################################################
@bot.callback_query_handler(func=lambda call: True)
def handle_callback_query(call):
    """ Method to handle the callback query """
    
    global offers_day
    
    if call.data == "opt1":
        bot.answer_callback_query(call.id, "Pesquisar Preço de Produto!")
    
    elif call.data == "opt2":
        
        offers_day = True
        
        bot.answer_callback_query(call.id, "Ofertas do Dia!")        
        departments = amazon_config.get_departaments_offer_of_the_day()
        
        departments_formatted = "\n".join(departments)
        
        bot.send_message(call.message.chat.id, f"Departamentos disponíveis:\n{departments_formatted}")
        bot.send_message(call.message.chat.id, "Qual departamento você deseja ver as ofertas?")
    
    elif call.data == "opt3":
        bot.answer_callback_query(call.id, "Prime Vídeo!")

################################################################################

#### Run Bot

In [4]:
bot.polling()

Error: Message: element click intercepted: Element is not clickable at point (90, 865)
  (Session info: MicrosoftEdge=131.0.2903.51)
Stacktrace:
	GetHandleVerifier [0x00007FF6A847DC75+12853]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6A87283C4+2250276]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6A86676A6+1460486]
	(No symbol) [0x00007FF6A827FB5C]
	(No symbol) [0x00007FF6A827DD19]
	(No symbol) [0x00007FF6A827BC3E]
	(No symbol) [0x00007FF6A827B041]
	(No symbol) [0x00007FF6A8270D4F]
	(No symbol) [0x00007FF6A82984FA]
	(No symbol) [0x00007FF6A8270617]
	(No symbol) [0x00007FF6A82704DD]
	(No symbol) [0x00007FF6A82987E0]
	(No symbol) [0x00007FF6A8270617]
	(No symbol) [0x00007FF6A82B1431]
	(No symbol) [0x00007FF6A8298163]
	(No symbol) [0x00007FF6A826FB54]
	(No symbol) [0x00007FF6A826F000]
	(No symbol) [0x00007FF6A826F741]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6A85FCDB4+1024020]
	Microsoft::Applications::Events::EventProperty

ReadTimeout: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=25)